In [1]:
import csv
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def get_dates(filename):
    #Facebook Messenger date format: Jun 8, 2013, 10:50 PM
    date_pattern = re.compile(r'[A-Z]{1}[a-z]{2}\s\d+[,]\s\d{4}[,]\s\d{1,2}[:]\d{2}\s[AM|PM]')
    with open(filename, mode='r', encoding='UTF-8', errors='ignore') as fp:
        soup = BeautifulSoup(fp, "lxml", from_encoding='utf-8')
        
        # delete the useless tags
        soup.find('head').decompose()
        soup.find('div', id="bluebarRoot").decompose()
        soup.find('p').decompose()
        for gen in soup.find_all(text=re.compile("^Generated")): # a tag which refers to the creation of the HTML file
            div = gen.find_parent("div")
            if div:  # saves a safety feature
                div.extract()
        for x in soup.find_all(lambda tag: not tag.name == "br"): # all the empty tags
            if len(x.text) == 0:
                x.extract()
        
        dates = soup.find_all(text=date_pattern)
        csvfile = csv.writer(open("chatdata.csv", "w", encoding='UTF-8'))
        csvfile.writerow(["date"])
        for date in dates:
            data = datetime.strptime(date,'%b %d, %Y, %I:%M %p')
            csvfile.writerow([data])

In [3]:
get_dates(r"D:\Acer\Downloads\facebook-pedromlsreis\messages\inbox\CarolinaConceicao_fNBJhpkbLw\message.html")

In [4]:
# get_dates(r"\Path\to\File\message.html")

In [5]:
df = pd.read_csv('chatdata.csv')
df['date'] = pd.to_datetime(df['date'])

L = ['year', 'month', 'day', 'hour', 'minute', 'dayofweek', 'dayofyear', 'weekofyear', 'quarter']
df = df.join(pd.concat((getattr(df['date'].dt, i).rename(i) for i in L), axis=1))

df['weekend'] = ((pd.DatetimeIndex(df['date']).dayofweek) // 5 == 1)
df.drop("date", axis=1, inplace=True)

df.head()

,year,month,day,hour,minute,dayofweek,dayofyear,weekofyear,quarter,weekend
0,2018,12,22,14,23,5,356,51,4,True
1,2018,12,21,2,35,4,355,51,4,False
2,2018,12,21,2,35,4,355,51,4,False
3,2018,12,21,2,35,4,355,51,4,False
4,2018,12,21,2,35,4,355,51,4,False
